# Examine & consume web services in Python asynchronously

                               ***Applies to: Machine Learning Server 9.2***

## 1. Authenticate and initiate the  `DeployClient`

There are several ways to authentication against Machine Learning Server. Choose the authentication method configured by your administrator. Contact your administrator for authentication credentials.

For simplicity, this example uses the local 'admin' account for authentication.  

1. Import the DeployClient and MLServer classes from the [azureml-model-management-sdk package](https://docs.microsoft.com/en-us/r-server/python-reference/azureml-model-management-sdk/azureml-model-management-sdk) to connect to Machine Learning Server (`use=MLServer`).

1. Provide **the connection details for your host and context** into the corresponding fields. Learn more in the article [Connecting to Machine Learning Server in Python](https://docs.microsoft.com/en-us/r-server/operationalize/python/how-to-authenticate-in-python).

In [1]:
# -- Import the DeployClient and MLServer classes from the azureml-model-management-sdk package.
from azureml.deploy import DeployClient
from azureml.deploy.server import MLServer

# -- Define the location of Machine Learning Server --
# -- for local onebox: http://localhost:12800
HOST = 'http://localhost:12800'
context = ('admin', 'Happiness_1')
client = DeployClient(HOST, use=MLServer, auth=context)

You are now authenticated. 

The **DeployClient** can interact with the web service management APIs to deploy, list, consume and so on. 

## 2. Create a sample web service to test batch

Create a web service now so we can consume it asynchronously next.

In normal conditions, the service would be ready for you to consume. For this sample, we'll create one.  This code follows the [Deploy a web service QuickStart](https://github.com/Microsoft/ML-Server-Python-Samples/blob/master/operationalize/Quickstart_Publish_Python_Web_Service.ipynb).  See this example for more context.


In [2]:
# Delete any existing service by this name.
client.delete_service('TxService', version='1.0')

# Read in the mtcars dataset you'll use when modeling
from microsoftml.datasets.datasets import DataSetMtCars
mtcars = DataSetMtCars()

# -- Represent the dataset as a dataframe.
mtcars = mtcars.as_df()

# Create and run a generalized linear model locally 
import pandas as pd
from revoscalepy import rx_lin_mod, rx_predict

cars_model = rx_lin_mod(
    formula='am ~ hp + wt',
    data=mtcars)

# Define an `init` function to handle service initialization
def init():
    import pandas as pd
    from revoscalepy import rx_predict

# Produce a prediction function called manualTransmission 
def manualTransmission(hp, wt):
    import pandas as pd
    from revoscalepy import rx_predict
    
    # -- make the prediction use model `cars_model` and input data --
    newData = pd.DataFrame({'hp':[hp], 'wt':[wt]})
    answer = rx_predict(cars_model, newData, type='response')
    
    # -- save some files to demonstrate the ability to return file artifacts --
    answer.to_csv('answer.csv')
    # return prediction
    return answer

# Publish the linear model as a Python web service with 
service_name = 'TxService'

service = client.service(service_name)\
        .version('1.0')\
        .code_fn(manualTransmission, init)\
        .inputs(hp=float, wt=float)\
        .outputs(answer=pd.DataFrame)\
        .models(cars_model=cars_model)\
        .description('My first python model')\
        .artifacts(['answer.csv'])\
        .deploy()

Rows Read: 32, Total Rows Processed: 32, Total Chunk Time: 0.001 seconds 
Computation time: 0.006 seconds.


## 3. Get the service you want to explore and consume

In a production environment, the data scientist who deployed the service would likely inform you that the new service is ready for testing or production. They should provide you with its name and version.

### 3.a.    Get a list of services

You can get a list of the services available on Machine Learning Server using the `list_services` function on the `DeployClient` object we initiated at the beginning of the notebook. [Learn more about list_services.](https://docs.microsoft.com/machine-learning-server/operationalize/python/how-to-consume-web-services#list_services)

In [3]:
# -- List all services by this name --
client.list_services('TxService')

[{'code': "def manualTransmission(hp, wt):\n    import pandas as pd\n    from revoscalepy import rx_predict\n    \n    # -- make the prediction use model `cars_model` and input data --\n    newData = pd.DataFrame({'hp':[hp], 'wt':[wt]})\n    answer = rx_predict(cars_model, newData, type='response')\n    \n    # -- save some files to demonstrate the ability to return file artifacts --\n    answer.to_csv('answer.csv')\n    # return prediction\n    return answer\n\nanswer = manualTransmission(hp, wt)",
  'creationTime': '2017-10-05T22:49:33.7832679',
  'description': 'My first python model',
  'initCode': 'def init():\n    import pandas as pd\n    from revoscalepy import rx_predict\n\ninit()',
  'inputParameterDefinitions': [{'name': 'hp', 'type': 'numeric'},
   {'name': 'wt', 'type': 'numeric'}],
  'myPermissionOnService': 'read/write',
  'name': 'TxService',
  'operationId': 'manualTransmission',
  'outputFileNames': ['answer.csv'],
  'outputParameterDefinitions': [{'name': 'answer', 't

You can now see all versions of TxService. There may be one or more. 

### 3.b.  Get the service object for TxService 1.0

You can retrieve the web service object for consumption by calling `get_service` on the `DeployClient` object. This object contains the client stub used to consume that service.

In [4]:
# -- Return the web service object for TxService 1.0 and assign to svc.
svc = client.get_service('TxService', version='1.0')
print(svc)

<TxserviceService> 
{   '_api': '/api/TxService/1.0',
    '_http_client': <azureml.common.http_client.HttpClient object at 0x00000060C1063780>,
    '_service': {   '_fn': {   'args': 'self,hp,wt',
                               'name': 'manualTransmission'},
                    'code': 'def '
                            'manualTransmission(hp, '
                            'wt):\n'
                            '    '
                            'import '
                            'pandas '
                            'as '
                            'pd\n'
                            '    '
                            'from '
                            'revoscalepy '
                            'import '
                            'rx_predict\n'
                            '    \n'
                            '    '
                            '# '
                            '-- '
                            'make '
                            'the '
                            'p

## 4. Explore the published web service

Now let's:
+ Use the `help` function to explore the published service. You can get help on any `azureml-model-management-sdk` functions, even those we dynamically generated ones to learn more about them.

+ Print the capabilities that define the service holdings to see what the service can do and how it should be consumed. [Learn more about capabilities...](https://docs.microsoft.com/machine-learning-server/python-reference/azureml-model-management-sdk/service#capabilities)

**Note:** Since you deployed the model as a web service, you can the code stored within a web service. However, unless you published the web service or are assigned to the "Owner" role, you won't be able to see the code contained in a service.

In [5]:
# -- Learn more about the service.
print(help(svc))

Help on TxserviceService in module azureml.deploy.server.service object:

class TxserviceService(Service)
 |  Service object from metadata.
 |  
 |  Method resolution order:
 |      TxserviceService
 |      Service
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, service, http_client)
 |      Constructor
 |      
 |      :param service:
 |      :param http_client:
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  batch(self, records, parallel_count=10)
 |      Register a set of input records for batch execution on this service.
 |      
 |      :param records: The `data.frame` or `list` of
 |             input records to execute.
 |      :param parallel_count: Number of threads used to process entries in
 |             the batch. Default value is 10. Please make sure not to use too
 |             high of a number because it might negatively impact performance.
 |      :return: The `Batch` object to control service batching
 |              lifecycle

In [6]:
# -- View the service object's capabilities/schema.
svc.capabilities()
# -- Notice the available public functions.

{'api': '/api/TxService/1.0',
 'artifacts': ['answer.csv'],
 'creation_time': '2017-10-05T22:49:33.7832679',
 'description': 'My first python model',
 'inputs': [{'name': 'hp', 'type': 'numeric'},
  {'name': 'wt', 'type': 'numeric'}],
 'inputs_encoded': [{'name': 'hp', 'type': 'float'},
  {'name': 'wt', 'type': 'float'}],
 'name': 'TxService',
 'operation_id': 'manualTransmission',
 'outputs': [{'name': 'answer', 'type': 'data.frame'}],
 'outputs_encoded': [{'name': 'answer', 'type': 'pandas.DataFrame'}],
 'public-functions': {'batch': 'batch(records, parallel_count=10)',
  'capabilities': 'capabilities()',
  'get_batch': 'get_batch(execution_id)',
  'list_batch_execution': 'list_batch_execution()',
  'manualTransmission': 'manualTransmission(self,hp,wt)',
  'swagger': 'swagger(json=True)'},
 'published_by': 'admin',
 'runtime': 'Python',
 'snapshot_id': 'dfe86727-c289-42f4-916d-3daa75f06d82',
 'swagger': 'http://localhost:12800/api/TxService/1.0/swagger.json',
 'version': '1.0'}

## 5. Consume the service asynchronously and make a prediction

Once the service is published, you can consume it using the _service api_ object returned from `get_service` to verify that the results are as expected. In this example, we will consume the service asynchronously in batch mode.

Generally speaking, the process for asynchronous batch consumption of a web service involves the following:

1. Call the web service on which the batch execution should be run

2. Define the data records for the batch execution task

3. Start (or cancel) the batch execution task

4. Monitor task and interact with results


Anyone can test and consume the service using its auto-generated Swagger-based JSON file. This Swagger-based JSON file is specific to a given version of a service. You can easily get this file during the same authenticated session in which you published the service or after the fact as will do here. It can be downloaded to the local file system.  **You can share the resulting file with application developers or others testing your service.**

### 5.a  Define the record data for the batch execution task. 
 
Record data comes from the mtcars dataset, which is a data.frame of 11 cols with names (mpg, cyl, ..., carb) and 32 rows of numerics. For simplicity, we just use two columns of the dataframe.

In [7]:
# -- Import the dataset from the microsoftml package
from microsoftml.datasets.datasets import get_dataset
mtcars = get_dataset('mtcars')

# -- Represent the dataset as a dataframe.
mtcars = mtcars.as_df()

# -- Define the data for the execution.
records = mtcars[['hp', 'wt']]

### 5.b  Assign the record data to the batch service and set the thread count. 

Use the public api function `batch` to define the input record data for the batch and set the number of concurrent threads for processing.

Syntax: `batch(inputs, parallelCount = 10)`

+ `inputs` specify the dataframe name directly.

+ `parallelCount` is the number of concurrent threads that can be dedicated to processing records in the batch. Take care not to set a number so high that it negatively impacts performance. Default is 10.

In [8]:
batch = svc.batch(records)

### 5.c  Start the batch service and its execution ID.

Use the public function `start()` to start the batch service.

After starting the batch execution, you can get the `execution_id` of the batch service so you can monitor the execution and interact with the results. Then, you can assign the returned results to a batch result object.

Once you have the batch object, use the execution_id public function to get the ID for the execution.

In [9]:
batch = batch.start()

# Get the batch execution id.
id = batch.execution_id

print("The execution_id of this batch service is {}".format(id))

The execution_id of this batch service is 10bebedb-9ee7-420d-ace3-71e3f194a6f5


### 5.d  Monitor task and interact with results

Using this id and the `results()` public function on the batch object, you can monitor batch execution results.  

In [10]:
# Check the results every second until the task finishes or fails. 
# Assign returned results to the 'batchres' batch result object.
import time

batchRes = None
while(True):
    batchRes = batch.results()
    print(batchRes)
    if batchRes.state == "Failed":
        print("Batch execution failed")  
        break
    if batchRes.state == "Complete": 
        print("Batch execution succeeded")  
        break
    print("Polling for asynchronous batch to complete...")
    time.sleep(1)


<BatchResponse> 
   api: /api/TxService/1.0 
   execution_id: 10bebedb-9ee7-420d-ace3-71e3f194a6f5
   completed_item_count : 32
   total_item_count: 32
   state: Complete
Batch execution succeeded


Once the batch task is complete, you can get the execution records by index from the batch results object, 'batchRes'. 

You can then access the result for each row of the input record data by passing index into the execution method.

In [11]:
for i in range(batchRes.completed_item_count):
    print("The result for {} th row in the record data is: {}".\
          format(i, batchRes.execution(i).outputs['answer']))

The result for 0 th row in the record data is:     am_Pred
0  0.592204
The result for 1 th row in the record data is:     am_Pred
0  0.469917
The result for 2 th row in the record data is:    am_Pred
0  0.68952
The result for 3 th row in the record data is:     am_Pred
0  0.306868
The result for 4 th row in the record data is:     am_Pred
0  0.376956
The result for 5 th row in the record data is:     am_Pred
0  0.175686
The result for 6 th row in the record data is:     am_Pred
0  0.506294
The result for 7 th row in the record data is:    am_Pred
0  0.18742
The result for 8 th row in the record data is:     am_Pred
0  0.296965
The result for 9 th row in the record data is:     am_Pred
0  0.234566
The result for 10 th row in the record data is:     am_Pred
0  0.234566
The result for 11 th row in the record data is:     am_Pred
0  0.088528
The result for 12 th row in the record data is:     am_Pred
0  0.251577
The result for 13 th row in the record data is:     am_Pred
0  0.227599
The re

## 5. Get list of generated artifacts

You can also access any artifacts generated for each row of the record data.


In [12]:
# List every artifact generated by this execution index for a specific row.
# Here, each row should have a "answer.csv" file and a "image.png" file.
lst_artifact = batch.list_artifacts(1)
print(lst_artifact)

# Then, get the contents of each artifact returned in the previous list.
# The result is a byte string of the corresponding object.
for obj in lst_artifact:
    content = batch.artifact(1, obj)

# Then, download the artifacts from execution index to the current working directory  
# unless a dest = "<path>" is specified.
# Here, these two files are in the working directory.
batch.download(1, "answer.csv")

['answer.csv']


['C:\\Program Files\\Microsoft\\ML Server\\PYTHON_SERVER\\Scripts\\answer.csv']